# Creating a Donkey vehicle

Puh, [The Donkey Convolutional Neural Network](./donkey-nn.ipynb) was a lot of one-way information. So, let's actually code something by creating our own `DonkeyCar` with a slightly changed `keras.py`.

The goals of this chapter are:
- To get an introduction to Donkey templates and parts
- To isolate the neural network by creating a new `Vehicle` with only code necessary to **train** the neural network. We'll use this knowledge in the later chapters to convert the training script to the [SageMaker SDK](https://github.com/aws/sagemaker-python-sdk).

## The Donkey library

As mentioned before, the [Donkey library](https://github.com/wroscoe/donkey) has several components.

It is first and foremost a python library installed where your other python libraries are (e.g. system python or virtualenv). After installation, you can `import` it as any normal python library:

```python
import donkeycar as dk
```

It also has a CLI with tools mainly used to aid training (see [donkey-tools.ipynb](./donkey-tools.ipynb)):

```bash
donkey --help
```

A `Vehicle` application, installed to the `~/d2` directory by default. This is where you'll find the `manage.py` script, which is used for both **driving** and **training**.

```bash
~/d2/manage.py --help
```

## The Donkey application

Also called the `Vehicle` or `Donkey` application and located in the `~/d2` directory.

Let's have a look at the **training** part of a Donkey `Vehicle` and `manage.py`. In short, this is what's going on:
1. When you install the `Vehicle` to the `~/d2` directory, it is per default installed from the [`donkeycar/templates/donkey2.py`](https://github.com/wroscoe/donkey/blob/master/donkeycar/templates/donkey2.py) template. It creates a `Vehicle` using different Donkey parts.
2. Creating `Donkey` parts is more of a focus in the [IoT track](../docs/PREPARE-IOT.md), so we will not dig too deep into what is going on (we leave that for you to sync within the team =)). Examples of parts are Joystick, motor driver and the ML model. See github for more parts: https://github.com/wroscoe/donkey/tree/master/donkeycar/parts
3. The installed `Vehicle` has 2 standard methods; `drive()` and `train()` and is invoked using the CLI. 
  - `drive()` can either be set to manual, and produce training data, or be given a pre-trained model to use when driving autonomously.
  - `train()` trains the model using the collected training data (*Tubs*). Usually done in SageMaker (see [previous chapter](./donkey-train.ipynb)) or on the host computer, which requires you to install the library there first.
  
In this chapter, we'll create a new, simple template for training only, and install it on the training machine (SageMaker or host).

## Create a new Donkey part

First, we'll create a copy of the model used in training as a new `Donkey` part. Create a file called `cnn.py` in the `donkeycar/parts/` by pasting the following (assuming you have `donkey` in you local directory):

In [ ]:
%%bash
# Create a new part.
cat > ~/SageMaker/donkey/donkeycar/parts/cnn.py << EOF
import keras
import donkeycar as dk


class KerasPilot():

    def load(self, model_path):
        self.model = keras.models.load_model(model_path)

    def shutdown(self):
        pass
    
    def train(self, train_gen, val_gen, 
              saved_model_path, epochs=100, steps=100, train_split=0.8,
              verbose=1, min_delta=.0005, patience=5, use_early_stop=True):

        save_best = keras.callbacks.ModelCheckpoint(saved_model_path, 
                                                    monitor='val_loss', 
                                                    verbose=verbose, 
                                                    save_best_only=True, 
                                                    mode='min')
        
        early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                   min_delta=min_delta, 
                                                   patience=patience, 
                                                   verbose=verbose, 
                                                   mode='auto')

        callbacks_list = [save_best]

        if use_early_stop:
            callbacks_list.append(early_stop)
        
        hist = self.model.fit_generator(
                        train_gen, 
                        steps_per_epoch=steps, 
                        epochs=epochs, 
                        verbose=1, 
                        validation_data=val_gen,
                        callbacks=callbacks_list, 
                        validation_steps=steps*(1.0 - train_split))
        return hist


class MyPilot(KerasPilot):
    def __init__(self, model=None, *args, **kwargs):
        super(MyPilot, self).__init__(*args, **kwargs)
        self.model = model if model else default_categorical()
        
    def run(self, img_arr):
        img_arr = img_arr.reshape((1,) + img_arr.shape)
        angle_binned, throttle = self.model.predict(img_arr)
        angle_unbinned = dk.utils.linear_unbin(angle_binned)
        return angle_unbinned, throttle[0][0]


def default_categorical():
    from keras.layers import Input, Dense, merge
    from keras.models import Model
    from keras.layers import Convolution2D, MaxPooling2D, Reshape, BatchNormalization
    from keras.layers import Activation, Dropout, Flatten, Dense
    
    img_in = Input(shape=(120, 160, 3), name='img_in')
    x = img_in
    x = Convolution2D(24, (5,5), strides=(2,2), activation='relu')(x)
    x = Convolution2D(32, (5,5), strides=(2,2), activation='relu')(x)
    x = Convolution2D(64, (5,5), strides=(2,2), activation='relu')(x)
    x = Convolution2D(64, (3,3), strides=(2,2), activation='relu')(x)
    x = Convolution2D(64, (3,3), strides=(1,1), activation='relu')(x)

    x = Flatten(name='flattened')(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(.1)(x)
    x = Dense(50, activation='relu')(x)
    x = Dropout(.1)(x)
    angle_out = Dense(15, activation='softmax', name='angle_out')(x)
    throttle_out = Dense(1, activation='relu', name='throttle_out')(x)
    
    model = Model(inputs=[img_in], outputs=[angle_out, throttle_out])
    model.compile(optimizer='adam',
                  loss={'angle_out': 'categorical_crossentropy', 
                        'throttle_out': 'mean_absolute_error'},
                  loss_weights={'angle_out': 0.9, 'throttle_out': .001})

    return model

EOF

Nice. As you can see, we've basically just removed some code from `keras.py`. Reinstall the library:

In [ ]:
# Replace the path with path to where you cloned the donkey git 
!pip install ~/SageMaker/donkey

In [ ]:
%%bash
# Create new Donkey template.
cat > ~/SageMaker/donkey/donkeycar/templates/robolab.py << EOF
#!/usr/bin/env python3
"""
Scripts to train a donkey 2.

Usage:
    manage.py (train) [--tub=<tub1,tub2,..tubn>]  (--model=<model>)

Options:
    -h --help        Show this screen.
    --tub TUBPATHS   List of paths to tubs. Comma separated. Use quotes to use wildcards. ie "~/tubs/*"
"""
import os
from docopt import docopt
import donkeycar as dk

from donkeycar.parts.cnn import MyPilot
from donkeycar.parts.datastore import TubGroup


def train(cfg, tub_names, model_name):
    X_keys = ['cam/image_array']
    y_keys = ['user/angle', 'user/throttle']

    def rt(record):
        record['user/angle'] = dk.utils.linear_bin(record['user/angle'])
        return record

    kl = MyPilot()
    print('tub_names', tub_names)
    if not tub_names:
        tub_names = os.path.join(cfg.DATA_PATH, '*')
    tubgroup = TubGroup(tub_names)
    train_gen, val_gen = tubgroup.get_train_val_gen(X_keys, y_keys, record_transform=rt,
                                                    batch_size=cfg.BATCH_SIZE,
                                                    train_frac=cfg.TRAIN_TEST_SPLIT)

    model_path = os.path.expanduser(model_name)

    total_records = len(tubgroup.df)
    total_train = int(total_records * cfg.TRAIN_TEST_SPLIT)
    total_val = total_records - total_train
    print('train: %d, validation: %d' % (total_train, total_val))
    steps_per_epoch = total_train // cfg.BATCH_SIZE
    print('steps_per_epoch', steps_per_epoch)

    kl.train(train_gen,
             val_gen,
             saved_model_path=model_path,
             steps=steps_per_epoch,
             train_split=cfg.TRAIN_TEST_SPLIT,
             epochs=1) # <------ Run only 1 epoch


if __name__ == '__main__':
    args = docopt(__doc__)
    cfg = dk.load_config()
    
    if args['train']:
        tub = args['--tub']
        model = args['--model']
        train(cfg, tub, model)
EOF

Very nice. Now let's create a new car:

In [ ]:
# Create a new car using the new template
![ -d ~/d3 ] && rm -rfv ~/d3
!donkey createcar --template robolab --path ~/d3

In [ ]:
%%time

!cat manage.py

# Let's try the new car
%cd ~/d3
!python manage.py train --tub='../SageMaker/data/tub_8_18-02-09' --model '../SageMaker/models/my-test-model'
%cd ~/SageMaker

Sweet, seems to work. Now that we now how to create a new template and our own training part, we can improve the network in any way we like.

## Next

[Training in the cloud](./donkey-cloud-train.ipynb)